# deep_learning_model_for_national_exam.ipynb

This notebook analyzes student performance.

**Author:** Birhanu Moges


In [ ]:
import pandas as pd
import numpy as np

print("Notebook loaded successfully")